In [1]:
import re
import aocd
from dataclasses import dataclass
from typing import Callable, Self
from operator import lt, gt

In [2]:
@dataclass
class Part:
    x: int
    m: int
    a: int
    s: int
    
    @classmethod
    def parse(cls, string) -> Self:
        x, m, a, s = re.findall(r'\d+', string)
        return cls(int(x), int(m), int(a), int(s))

@dataclass
class Rule:
    condition: Callable[[int, int, int, int], bool]
    workflow: str

    @classmethod
    def parse(cls, string) -> Self:
        match string.split(':'):
            case cond, workflow:
                return cls(lambda x, m, a, s: eval(cond), workflow)
            case [workflow]:
                return cls(lambda x, m, a, s: True, workflow)
        raise Exception("could not parse rule")
        

@dataclass
class Workflow:
    name: str
    rules: list[Rule]

    def consider(self, part: Part, workflows: dict[str, Self]) -> str:
        for rule in self.rules:
            if rule.condition(part.x, part.m, part.a, part.s):
                if rule.workflow in workflows.keys():
                    return workflows[rule.workflow].consider(part, workflows)
                return rule.workflow
        raise Exception("no workflow")

    @classmethod
    def parse(cls, string) -> Self:
        if m := re.match(r'(\w+){(.+)}', string):
            name = m[1]
            rules = [Rule.parse(s) for s in m[2].split(',')]
            return cls(name, rules)
        raise Exception("could not parse workflow")


In [3]:
data = \
"""px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

data = aocd.get_data(day=19, year=2023)


In [4]:
workflow_lines, part_lines = [chunk.splitlines() for chunk in data.split('\n\n')]
parts = [Part.parse(line) for line in part_lines]
workflows = {w.name: w for w in (Workflow.parse(line) for line in workflow_lines)}

In [5]:
s = 0
for part in parts:
    if workflows['in'].consider(part, workflows) == 'A':
        s += part.x + part.m + part.a + part.s

print("Part 1:", s)

Part 1: 376008


In [6]:
workflows

{'vr': Workflow(name='vr', rules=[Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abfef20>, workflow='A'), Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abfefc0>, workflow='R')]),
 'lx': Workflow(name='lx', rules=[Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff060>, workflow='A'), Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff100>, workflow='A')]),
 'vkm': Workflow(name='vkm', rules=[Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff1a0>, workflow='A'), Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff240>, workflow='R'), Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff2e0>, workflow='A')]),
 'nr': Workflow(name='nr', rules=[Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff380>, workflow='R'), Rule(condition=<function Rule.parse.<locals>.<lambda> at 0x10abff420>, workflow='A')]),
 'xd': Workflow(name='xd', rules=[Rule(condition=<function Rule.parse.<locals>.<